In [ ]:
import cv2
import itertools as it
import numpy as np
import pickle

def show_and_wait(frame, title='tesst', wait_time=0):
    cv2.imshow(title, frame)
    key = cv2.waitKey(wait_time)
    return None if key < 0 else chr(key)

def read_and_wait(video, title='tesst', wait_time=0):
    result, frame = video.read()
    if result:
        return show_and_wait(frame, title, wait_time)

In [ ]:
file_path = r"C:\Users\cidzerda\Documents\GitHub\ctc\hero_names.txt"
with open(file_path) as fin:
    hero_names = [s.strip() for s in fin]
len(hero_names)

In [ ]:
all_letters = set(chr(i + ord('A')) for i in range(26))
def get_letters(hero_names):
    g = it.chain.from_iterable(hero_names)
    s = set(c for c in g if 'A' <= c <= 'Z')
    return all_letters - s
def get_letter(letters, available_hero_names):
    def count_names(c):
        l = [s for s in available_hero_names if c in s]
        return len(l)
    d = {count_names(c): c for c in letters}
    n = min(d)
    return d[n]
def get_names(letter, available_hero_names):
    return [s for s in available_hero_names if letter in s]
def get_name(l, letters):
    def count_letters(s):
        s = letters - set(s)
        return len(s)
    d = {count_letters(s): s for s in l}
    n = min(d)
    return d[n]
def fn(hero_names):
    # Create a list of selected hero names.
    selected_hero_names = []
    # Create a list of available hero names.
    available_hero_names = list(hero_names)
    # Create a set of accounted letters.
    accounted_letters = set()
    while len(accounted_letters) < 26:
        # Create a list of letters that are not represented in the list of selected names.
        letters = all_letters - accounted_letters
        # From that list, select the letter that is in the fewest names of those in the available names.
        letter = get_letter(letters, available_hero_names)
        # Get the list of names that contain that letter and are in the available names.
        l = get_names(letter, available_hero_names)
        # From that list, select the name with the greatest number of letters that are
        # not represented in the list of selected names.
        s = get_name(l, letters)
        # Add that name to the list of selected names.
        selected_hero_names.append(s)
        # Add that name's letters to the set of accounted letters.
        accounted_letters.update(c for c in s if 'A' <= c <= 'Z')
    return selected_hero_names
print(fn(hero_names))

In [ ]:
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\tmp\2020-01-15-grandgrant.mp4")
print(video.get(cv2.CAP_PROP_FRAME_COUNT))
video.set(cv2.CAP_PROP_POS_MSEC, (19 * 60 + 21) * 1000) # game start
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (22 * 60 + 56) * 1000) # before UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (23 * 60 + 15) * 1000) # after UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (25 * 60 +  0) * 1000) # before pause
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (25 * 60 + 50) * 1000) # after pause
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (27 * 60 + 59) * 1000) # before UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (28 * 60 + 11) * 1000) # after UI change
read_and_wait(video)
video.set(cv2.CAP_PROP_POS_MSEC, (53 * 60 + 34) * 1000) # game end
read_and_wait(video)
video.release()
cv2.destroyAllWindows()

In [ ]:
ranges = (
    (19 * 60 + 21, 22 * 60 + 56),
    (23 * 60 + 15, 25 * 60 +  0),
    (25 * 60 + 50, 27 * 60 + 59),
    (28 * 60 + 11, 53 * 60 + 34),
)
video = cv2.VideoCapture(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\2020-01-15-grandgrant.mp4")
for begin, end in ranges:
    print(begin)
    for i in range(begin, end):
        video.set(cv2.CAP_PROP_POS_MSEC, i * 1000)
        _, frame = video.read()
        ch = show_and_wait(frame, wait_time=25)
        if ch == 'q':
            break
        if ch == ' ':
            print(video.get(cv2.CAP_PROP_POS_MSEC))
            input()
video.release()
cv2.destroyAllWindows()

In [ ]:
ranges = (
    (19 * 60 + 21, 22 * 60 + 56),
    (23 * 60 + 15, 25 * 60 +  0),
    (25 * 60 + 50, 27 * 60 + 59),
    (28 * 60 + 11, 53 * 60 + 34),
)
print(sum(end - begin for begin, end in ranges))

In [ ]:
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.txt") as fin:
    labels =  fin.readline()
with open(r"C:\Users\cidzerda\Documents\GitHub\strevr\data\selected.pickle", 'rb') as fin:
    images = [pickle.load(fin) for _ in labels]

# Move invalid images to a separate array.
invalid_images = images[-58:-54] + images[-45:-41] + images[-11:] # There are nineteen of these.
del images[-58:-54]
del images[-45:-41]
del images[-11:]

# Combine the list of three-dimensional tensors into a single four-dimensional tensor.
images = np.stack(images, axis=0)

# There are 2981 valid images.  That number's prime factors are 11 and 271.
print(images.shape)

In [ ]:
def fn(color, mask):
    background = np.stack([74/255 * np.ones(mask.shape[:2]), 172/255 * np.ones(mask.shape[:2]), 133/255 * np.ones(mask.shape[:2])], axis=-1)
    return background * mask + color * (1.0 - mask)

# Determine the desired color by computing the mean of all colors.
color = np.mean(images, axis=0)

# Determine the opacity mask by scaling the range of standard deviations to [0, 1].
std = np.std(images, axis=0)
mask = (std - np.min(std, axis=(0, 1))) / (np.max(std, axis=(0, 1)) - np.min(std, axis=(0, 1)))

print(std.shape)
print('min', np.min(std), 'max', np.max(std), 'mean', np.mean(std))
print('min', np.min(std, axis=(0, 1)), 'max', np.max(std, axis=(0, 1)), 'mean', np.mean(std, axis=(0, 1)))
show_and_wait(cv2.resize(np.vstack([fn(color, mask), images[0], images[99], images[222]]), (520, 240)))

# Verify "blue" is the first of the three values in a pixel tuple.
args = ((1, 1), np.float32)
blue = np.stack([np.ones(*args), np.zeros(*args), np.zeros(*args)], axis=-1)
print(blue.shape, blue)
show_and_wait(cv2.resize(blue, tuple(np.array(args[0]) * 99)))
print(cv2.cvtColor(blue, cv2.COLOR_BGR2Luv))
print(cv2.cvtColor(blue, cv2.COLOR_RGB2Luv))

# Manually compute L*u*v* assuming R=G=0, B=1
RGB = np.array([[0], [0], [1]])
M = np.array([[.412453, .35758, .180423], [.212671, .71516, .072169], [.019334, .119193, .950227]])
X, Y, Z = np.matmul(M, RGB)
L = 116*(Y**(1/3))-16 if Y > .008856 else 903.3*Y
u, v = 4*X/(X+15*Y+3*Z), 9*Y/(X+15*Y+3*Z)
u, v = 13*L*(u-.19793943),13*L*(v-.46831096)
print(L, u, v) # This output matches that of using COLOR_BGR2Luv above.

cv2.destroyAllWindows()